# Exploratory Data Analysis — France Grid Stress Prediction (Day-Ahead, D+1)


## Notebook objectives


- Understand the temporal structure of national electricity load and generation  
- Validate data integrity (time axis, missingness, DST, outliers)  
- Analyze demand-side (consumption) and supply-side (production) separately  
- Translate EDA findings into clear modeling decisions for day-ahead forecasting  


# 0. Executive overview


Ensuring the Project Root as the Working Directory


In [1]:
import os  # OS utilities for working with directories

# If the notebook is executed from the notebooks/ folder,
# move up one level to set the project root as the working directory
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Print the current working directory for verification
print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /home/onyxia/work/france-grid-stress-prediction


## 0.1 Dataset snapshot

- Period covered (start / end)
- Inferred frequency (hourly / half-hourly)
- Number of rows and columns
- Key variables detected (load, temperature, wind, solar, nuclear, hydro, thermal, etc.)


0.1.1 Imports & paths (raw data)

In [7]:
# --- 0.1.1 Imports & paths (raw data) ---
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path(".").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"

CONS_DIR = DATA_RAW / "consommation"                 # yearly folders: 1996/, 1997/, ...
WEATHER_DIR = DATA_RAW / "weather"                   # csv files: weather_32_cities_2015.csv, ...

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CONS_DIR exists:", CONS_DIR.exists(), "|", CONS_DIR)
print("WEATHER_DIR exists:", WEATHER_DIR.exists(), "|", WEATHER_DIR)


PROJECT_ROOT: /home/onyxia/work/france-grid-stress-prediction
CONS_DIR exists: True | /home/onyxia/work/france-grid-stress-prediction/data/raw/consommation
WEATHER_DIR exists: True | /home/onyxia/work/france-grid-stress-prediction/data/raw/weather


0.1.2 Helper functions (datetime detection, frequency inference, variable detection)

In [8]:
# --- 0.1.2 Helper functions (snapshot utilities) ---

def _normalize_cols(cols) -> list[str]:
    """Lowercase + strip column names for robust matching."""
    return [str(c).strip().lower() for c in cols]

def infer_freq_from_datetime(dt: pd.Series) -> str:
    """
    Infer dataset frequency from datetime differences (mode of diffs).
    Returns: 'half-hourly', 'hourly', or 'irregular/unknown'.
    """
    dt = pd.to_datetime(dt, errors="coerce").dropna().sort_values()
    if len(dt) < 3:
        return "irregular/unknown"
    diffs = dt.diff().dropna()
    if diffs.empty:
        return "irregular/unknown"

    # Most common time delta
    mode_delta = diffs.value_counts().idxmax()

    if mode_delta == pd.Timedelta(minutes=30):
        return "half-hourly"
    if mode_delta == pd.Timedelta(hours=1):
        return "hourly"
    return f"irregular/unknown (mode diff: {mode_delta})"

def detect_key_variables(columns: list[str]) -> dict[str, list[str]]:
    """
    Detect key variables from column names (best-effort).
    Returns a dict {category: [matching_columns]}.
    """
    cols = _normalize_cols(columns)

    patterns = {
        "load / consumption": ["cons", "load", "consommation", "puissance", "demande"],
        "temperature": ["temp", "temperature", "t2m"],
        "wind": ["wind", "eol", "eolien", "windspeed"],
        "solar": ["solar", "pv", "solaire", "irradiance", "radiation"],
        "nuclear": ["nuclear", "nucle", "nuclé", "nuc"],
        "hydro": ["hydro", "hydr", "barrage"],
        "thermal": ["thermal", "therm", "fossil", "gaz", "gas", "coal", "charbon", "fioul", "oil"],
        "imports/exports": ["import", "export", "exchange", "solde", "interconnexion"],
    }

    out = {k: [] for k in patterns.keys()}
    for original, c in zip(columns, cols):
        for cat, keys in patterns.items():
            if any(k in c for k in keys):
                out[cat].append(original)

    # Drop empty categories
    out = {k: v for k, v in out.items() if len(v) > 0}
    return out

def try_build_datetime(df: pd.DataFrame) -> pd.Series | None:
    """
    Build a datetime series from common patterns:
    - a direct datetime column (datetime/date/horodate/timestamp)
    - separate date + time/hour columns (date + heure/hour/time)
    Returns a pd.Series or None if not found.
    """
    cols = list(df.columns)
    cols_l = _normalize_cols(cols)

    # 1) Direct datetime-like column
    direct_candidates = [
        "datetime", "dateheure", "date_heure", "horodate", "timestamp",
        "date", "date_time", "time"
    ]
    for cand in direct_candidates:
        if cand in cols_l:
            s = pd.to_datetime(df[cols[cols_l.index(cand)]], errors="coerce")
            # Heuristic: accept if at least some datetimes parsed
            if s.notna().mean() > 0.5:
                return s

    # 2) Separate date + time/hour
    date_candidates = ["date", "jour"]
    time_candidates = ["heure", "hour", "time", "heures"]

    date_col = None
    time_col = None

    for dc in date_candidates:
        if dc in cols_l:
            date_col = cols[cols_l.index(dc)]
            break
    for tc in time_candidates:
        if tc in cols_l:
            time_col = cols[cols_l.index(tc)]
            break

    if date_col is not None and time_col is not None:
        d = pd.to_datetime(df[date_col], errors="coerce")
        t = df[time_col].astype(str).str.strip()

        # Common case: "0", "1", ... hours OR "00:30", "01:00", ...
        # Normalize pure integers into HH:MM
        t_norm = t.where(t.str.contains(":"), t.str.zfill(2) + ":00")
        s = pd.to_datetime(d.dt.strftime("%Y-%m-%d") + " " + t_norm, errors="coerce")
        if s.notna().mean() > 0.5:
            return s

    return None

def snapshot_df(df: pd.DataFrame, name: str) -> dict:
    """Compute a quick snapshot from an already-loaded dataframe."""
    dt = try_build_datetime(df)
    period = (pd.NaT, pd.NaT)
    freq = "irregular/unknown"
    if dt is not None:
        period = (pd.to_datetime(dt, errors="coerce").min(), pd.to_datetime(dt, errors="coerce").max())
        freq = infer_freq_from_datetime(dt)

    return {
        "dataset": name,
        "start": period[0],
        "end": period[1],
        "frequency": freq,
        "rows": int(df.shape[0]),
        "cols": int(df.shape[1]),
        "key_variables": detect_key_variables(list(df.columns)),
        "columns": list(df.columns),
    }


0.1.3 File discovery (consumption years + weather CSVs)

In [9]:
# --- 0.1.3 File discovery (raw files) ---

# Consumption files: e.g. data/raw/consommation/1996/Historique_consommation_INST_1996.xls
cons_files = sorted(CONS_DIR.glob("*/*Historique_consommation_INST_*.xls*"))

# Weather files: e.g. data/raw/weather/weather_32_cities_2015.csv, weather_32_cities_historical_2012.csv
weather_files = sorted(WEATHER_DIR.glob("weather_32_cities*.csv"))

print(f"Found {len(cons_files)} consumption files.")
print("Example:", cons_files[0] if cons_files else None)

print(f"Found {len(weather_files)} weather files.")
print("Examples:", weather_files[:3])


Found 30 consumption files.
Example: /home/onyxia/work/france-grid-stress-prediction/data/raw/consommation/1996/Historique_consommation_INST_1996.xls
Found 27 weather files.
Examples: [PosixPath('/home/onyxia/work/france-grid-stress-prediction/data/raw/weather/weather_32_cities_2015.csv'), PosixPath('/home/onyxia/work/france-grid-stress-prediction/data/raw/weather/weather_32_cities_2016.csv'), PosixPath('/home/onyxia/work/france-grid-stress-prediction/data/raw/weather/weather_32_cities_2017.csv')]


0.1.4 Snapshot: consumption (1996–2025) —  “per-year” scan

This scans each yearly file, extracts only a minimal sample to infer columns + datetime, and aggregates start/end across all years.

do: pip install xlrd>=2.0.1

In [12]:
# --- 0.1.4 Snapshot: consumption (1996–2025) ---

from collections import Counter

cons_files = sorted(CONS_DIR.glob("*/*Historique_consommation_INST_*.xls*"))

global_start, global_end = pd.NaT, pd.NaT
all_columns = set()
freq_counter = Counter()

for fp in cons_files:
    df = pd.read_excel(fp, nrows=5000)  # xlrd is available

    all_columns.update(df.columns.astype(str))

    dt = try_build_datetime(df)
    if dt is None:
        continue

    dt = pd.to_datetime(dt, errors="coerce").dropna()
    if dt.empty:
        continue

    global_start = dt.min() if pd.isna(global_start) else min(global_start, dt.min())
    global_end = dt.max() if pd.isna(global_end) else max(global_end, dt.max())
    freq_counter[infer_freq_from_datetime(dt)] += 1

cons_snapshot = {
    "dataset": "Consumption (raw, yearly files)",
    "start": global_start,
    "end": global_end,
    "frequency": freq_counter.most_common(1)[0][0],
    "n_files": len(cons_files),
    "n_columns_detected": len(all_columns),
    "key_variables": detect_key_variables(sorted(all_columns)),
}

cons_snapshot


IndexError: list index out of range

0.1.5 Snapshot: weather (historical + modern)

In [13]:
# --- 0.1.5 Snapshot: weather (historical + modern) ---

from collections import Counter

def summarize_weather(files: list[Path], label: str) -> dict:
    global_start, global_end = pd.NaT, pd.NaT
    all_columns = set()
    freq_counter = Counter()

    for fp in files:
        df = pd.read_csv(fp, nrows=20000)

        all_columns.update(df.columns.astype(str))

        dt = try_build_datetime_weather(df)
        if dt is None:
            continue

        dt = pd.to_datetime(dt, errors="coerce").dropna()
        if dt.empty:
            continue

        global_start = dt.min() if pd.isna(global_start) else min(global_start, dt.min())
        global_end = dt.max() if pd.isna(global_end) else max(global_end, dt.max())
        freq_counter[infer_freq_from_datetime(dt)] += 1

    return {
        "dataset": label,
        "start": global_start,
        "end": global_end,
        "frequency": freq_counter.most_common(1)[0][0],
        "n_files": len(files),
        "n_columns_detected": len(all_columns),
        "key_variables": detect_key_variables(sorted(all_columns)),
    }


weather_files = sorted(WEATHER_DIR.glob("weather_32_cities*.csv"))
weather_historical = [f for f in weather_files if "historical" in f.name.lower()]
weather_modern = [f for f in weather_files if "historical" not in f.name.lower()]

weather_hist_snapshot = summarize_weather(weather_historical, "Weather (historical, 1996–2009)")
weather_modern_snapshot = summarize_weather(weather_modern, "Weather (modern, 2015–2025)")

weather_hist_snapshot, weather_modern_snapshot


NameError: name 'try_build_datetime_weather' is not defined

## 0.2 Top data issues


Summarize any critical issues:
- Missing timestamp blocks
- DST anomalies (23h / 25h days)
- High missing-rate variables
- Extreme or suspicious values


## 0.3 Key findings


Write 3–5 high-level findings in plain language after completing the EDA.


## 0.4 Weekly mini-dashboard


Single figure over a representative week:
- Load
- Temperature
- Wind generation
- Solar generation


# 1. Data loading and reproducibility


## 1.1 Data sources


Document:
- RTE : 
https://www.services-rte.com/en/download-data-published-by-rte.html?category=consumption&type=power_consumption

- Weather sources : 
https://open-meteo.com/

- Calendar data (holidays, school vacations if used)


## 1.2 Load processed dataset


Load the consolidated dataset.
Show head / tail and basic shape.


## 1.3 Structural sanity checks


Check:
- Datetime index type and timezone
- Sorted and unique timestamps
- Inferred frequency
- Column naming consistency and units
- Coverage (min / max dates)


# 2. Time axis integrity (global)


## 2.1 Frequency audit


Analyze datetime differences:
- Distribution of time deltas
- Abnormal steps
- Largest gaps


## 2.2 Missing timestamps


- Build the expected full time grid
- Count missing timestamps
- Identify contiguous missing blocks (start / end / duration)


## 2.3 Daylight Saving Time (DST)


- Identify 23-hour and 25-hour days
- Decide and document the timezone policy


# 3. Data quality and anomalies (global)


## 3.1 Missing values analysis


- Missing rate per variable
- Missingness over time (year / month)
- Optional heatmap


## 3.2 Outlier detection


- Extreme values in load and generation
- Negative or impossible values
- Flatlines and step changes


## 3.3 Known event windows


Inspect known periods:
- COVID shock
- Extreme cold spells
- Other major system events (if known)


# Part A — Consumption (demand side)


# 4. National electricity load: global behavior


## 4.1 Long-term trend


- Multi-year load evolution
- Rolling averages
- Non-stationarity discussion


## 4.2 Load distribution and tails


- Histogram / density
- Quantiles
- Extreme demand days


# 5. Intra-day and intra-week patterns (load)


## 5.1 Daily profile


- Average load by hour
- Weekday vs weekend comparison


## 5.2 Weekly structure


- Mean load by weekday
- Hourly profiles for each weekday


# 6. Seasonal patterns (load)


## 6.1 Monthly seasonality


- Monthly averages
- Monthly boxplots


## 6.2 Year-to-year comparison


- Same-month comparisons across years
- Structural break detection


# 7. Thermo-sensitivity analysis


## 7.1 Load vs temperature


- Scatter plots
- Nonlinear smoothing
- Winter vs summer asymmetry


## 7.2 Degree-days


- Heating Degree Days (HDD)
- Cooling Degree Days (CDD)
- Comparison with raw temperature


## 7.3 Thermal inertia


- Lagged or smoothed temperature features


# Part B — Production (supply side)


# 8. Electricity generation: overview


## 8.1 Total generation vs load


- Compare total generation and load
- Identify stress periods (imports/exports if available)


## 8.2 Generation mix


- Average mix by technology
- Mix evolution over time


# 9. Production by technology


## 9.1 Nuclear


- Stability
- Seasonal modulation
- Ramp constraints


## 9.2 Wind


- Volatility
- Seasonal behavior
- Ramp rates


## 9.3 Solar


- Diurnal cycle
- Seasonal amplitude


## 9.4 Hydro and thermal


- Dispatchable role
- Response during stress periods


# 10. Weather dependence (production)


## 10.1 Wind vs wind speed


- Scatter plots
- Binned averages


## 10.2 Solar vs radiation


- Scatter plots
- Solar vs hour of day


## 10.3 Seasonal weather effects


- Winter vs summer comparisons


# 11. Residual load and grid stress


## 11.1 Definition


Residual load = load − (wind + solar)


## 11.2 Residual load analysis


- Distribution
- Daily profile
- Extreme peaks


## 11.3 Comparison with raw load


- Tail behavior
- Interpretation for grid stress


# Cross-cutting analysis


# 12. Calendar effects


## 12.1 Weekday effects


- Mean differences by weekday
- Interaction with seasonality


## 12.2 Holidays and vacations


- Holiday vs non-holiday comparison
- Vacation-period effects


# 13. Temporal dependence


## 13.1 Autocorrelation


- ACF / PACF for load and residual load


## 13.2 Lag correlations


- Correlation at 24h, 48h, 168h


# 14. Feature relationships


## 14.1 Correlation heatmaps


- Load, weather, generation variables


## 14.2 Redundancy checks


- Multicollinearity inspection


# 15. EDA synthesis and modeling decisions


## 15.1 Main conclusions


Summarize confirmed patterns:
- Seasonality
- Nonlinearity
- Intermittency
- Temporal dependence


## 15.2 Modeling implications


- Target choice
- Feature set
- Data cleaning policy
- Backtesting strategy


# Appendix
